In [0]:
# !pip install sklearn.cross_validation
import cv2
import os;

import keras
import numpy as np
from keras import Sequential, Input, Model
from keras.engine import InputLayer
from keras.layers import MaxPooling2D, Dropout, Flatten, Dense, Activation, Conv2D, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras_preprocessing.image import ImageDataGenerator

base_path = "drive/My Drive/Colab Notebooks"
train_path = base_path+"/train_mhsma/"
test_path = base_path+"/test_mhsma"
model_name = "update_model_copy.h5"
shape_size = 71
num_class = 4
def train_data_set(path):
    data_set = []
    for dir in os.listdir(path):
        new_path = os.path.join(path,dir)
        print(dir)
        for file in os.listdir(new_path):
            file_path = os.path.join(new_path,file)
            img = cv2.imread(file_path)
#             img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img,(shape_size,shape_size))
            data_set.append([np.array(img),dir])
    return data_set
  
##reading training data

train_data = train_data_set(train_path)

train_data_img = np.array([i[0] for i in train_data]).reshape(-1,shape_size,shape_size,3).astype("float32")
train_data_label = np.array([i[1] for i in train_data]).reshape(-1,1)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data_img, train_data_label, test_size = 0.1)





1
0


In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# convert training set shape according to num_class
y_train = keras.utils.to_categorical(y_train, num_classes=num_class)
y_test = keras.utils.to_categorical(y_test, num_classes=num_class)

In [0]:
print(x_train.shape)

(900, 71, 71, 3)


In [0]:


## build CNN 
def build_model(shape):
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
                   input_shape=shape))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  
  
  model.add(Conv2D(32, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

   
  model.add(Conv2D(32, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  
  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_class))
  model.add(Activation('softmax'))
  return model

timizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
import os
model_path = os.path.join(base_path,model_name)
from keras.models import load_model

## load model from model_path
if os.path.exists(model_path):
    print('using existing model')
    model = load_model(model_path)
else:
    model = build_model((71,71,3))


using existing model


In [0]:
## compile and eveluate model
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
print(model.evaluate(x_train,y_train,batch_size=300,verbose=0))


[8.706493695576986, 0.055555557211240135]


# Compile Model and build image data generator


In [0]:
## generator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format="channels_last",
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.2)
    
        # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).


# Training Model

In [0]:
##training 
datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=10),
                    steps_per_epoch=300,
                    epochs=300,
                    validation_data=(x_test,y_test),
                    shuffle=True,
                    workers=4)

# model.save(os.path.join(base_path,model_name)) 

Epoch 1/300
300/300 [==============================] - 10s 32ms/step - loss: 0.9031 - acc: 0.5307 - val_loss: 0.6811 - val_acc: 0.5900
Epoch 2/300
300/300 [==============================] - 8s 26ms/step - loss: 0.7036 - acc: 0.5660 - val_loss: 0.6732 - val_acc: 0.6000
Epoch 3/300
300/300 [==============================] - 8s 25ms/step - loss: 0.6818 - acc: 0.6050 - val_loss: 0.6694 - val_acc: 0.6700
Epoch 4/300
300/300 [==============================] - 8s 26ms/step - loss: 0.6632 - acc: 0.6243 - val_loss: 0.6494 - val_acc: 0.6200
Epoch 5/300
300/300 [==============================] - 8s 26ms/step - loss: 0.6613 - acc: 0.6313 - val_loss: 0.6482 - val_acc: 0.6300
Epoch 6/300
300/300 [==============================] - 8s 26ms/step - loss: 0.6506 - acc: 0.6250 - val_loss: 0.6420 - val_acc: 0.6600
Epoch 7/300
300/300 [==============================] - 8s 26ms/step - loss: 0.6468 - acc: 0.6383 - val_loss: 0.6479 - val_acc: 0.6400
Epoch 8/300
300/300 [==============================] - 8s 26m

# Save Model

In [0]:
print(base_path)
model.save(os.path.join(base_path,model_name)) 

drive/My Drive/Colab Notebooks


Tesing and printing the predicting results

In [0]:
dir_path = os.path.join(test_path,'01_Normal')
for fi in os.listdir(dir_path):
  img = cv2.imread(os.path.join(dir_path,fi))
  img = cv2.resize(img,(71,71))
  imgs =np.array([img])
#   print(img.shape)
  pred = model.predict(imgs)
  print(np.argmax(pred))

FileNotFoundError: ignored

In [0]:
print(pred)
print(np.argmax(pred))

In [0]:
dir_path = os.path.join(test_path,'01_Normal')
igg = cv2.imread(os.path.join(dir_path,'1_201102_2_i2_j2_k64.tiff3.tiff'))
print(igg.shape)

In [0]:
test_path = base_path+"/train_v5/p2/0"
shape_size=71
test_data_img = []
test_data_label = []
for i in os.listdir(test_path):
  img = cv2.imread(os.path.join(test_path,i))
  img = cv2.resize(img,(shape_size,shape_size))
  test_data_img.append(img)
  test_data_label.append(0)
test_dhttps://colab.research.google.com/drive/1jh0_Py4SokY8Cgka0dltH9Rw26cSkWAJ?authuser=1#ata_img = np.array(test_data_img).reshape(-1,shape_size,shape_size,3)
test_data_label = np.array(test_data_label).reshape(-1,1)

In [0]:
test_data_label = keras.utils.to_categorical(test_data_label, num_classes=num_class)
# y_test = keras.utils.to_categorical(y_test, num_classes=num_class)
print(model.evaluate(test_data_img,test_data_label,batch_size=300,verbose=0))

[0.5594710111618042, 0.8333333134651184]


Testing  

In [0]:
test_path = base_path+"/test_mhsma"
test_data = train_data_set(test_path)

test_data_img = np.array([i[0] for i in test_data]).reshape(-1,shape_size,shape_size,3).astype("float32")
test_data_label = np.array([i[1] for i in test_data]).reshape(-1,1)


1
0


In [0]:

print(test_data_img.shape)
print(test_data_label.shape)
print(len(test_data))

(300, 71, 71, 3)
(300, 1)
300


In [0]:

model_name = "update_model_copy.h5"
import os
base_path = "drive/My Drive/Colab Notebooks"
# setup_to_finetune(model)
model_path = os.path.join(base_path,model_name)
from keras.models import load_model
model = load_model(model_path)
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
test_data_label = keras.utils.to_categorical(test_data_label, num_classes=4)
# y_test = keras.utils.to_categorical(y_test, num_classes=num_class)


In [0]:
print(model.evaluate(test_data_img,test_data_label,batch_size=300,verbose=0))


[0.6450104713439941, 0.6733333468437195]
